In [1]:
import psycopg2
from psycopg2 import sql
from psycopg2 import Error

In [2]:
DB_HOST = 'pavlov.ing.puc.cl'
DB_PORT = '5432'
DB_USER = 'grupo8'
DB_PASSWORD = 'bulla8'
DB_NAME = 'grupo8e2'

In [3]:
#Muestre todos los restaurantes que ofrezcan un plato específico (string) ingresado y que esté disponible
def consulta1(): 
    #query devuelve nombre y sucursal de los restaurantes que ofrecen el plato ingresado
    query = """SELECT Restaurant.id, Restaurant.nombre, Restaurant.sucursal 
        FROM Plato
        JOIN Platorestaurant ON Plato.id = Platorestaurant.plato_id 
        JOIN Restaurant ON Platorestaurant.restaurant_id = Restaurant.id
        WHERE Plato.nombre = %(nombre)s 
        AND Plato.disponibilidad = TRUE 
        AND Restaurant.vigente = TRUE"""
    return query

#Muestre todos los pedidos de un usuario específico ingresado (email string) y su gasto mensual (solo los pedidos concretados)
def consulta2():
    query = """SELECT * 
        FROM Pedido
        JOIN Usuario ON Pedido.usuario_id = Usuario.id
        WHERE Usuario.email = %(email)s
        AND Pedido.fecha BETWEEN %(fecha_inicio)s AND %(fecha_fin)s"""
    
    query1 = """ SELECT Usuario.email,  (SUM(Empresa.precio_mensual) + SUM(Empresa.precio_anual / 12) + gasto_mensual_sin_suscripcion.gasto_mensual) as gasto_mensual 
            FROM Usuario, Suscripcion, Empresa, (SELECT Usuario.email as email_usuario, (SUM(gasto_comida.precio_comida) + SUM(gasto_comida.precio_despacho)) as gasto_mensual
                                        FROM Usuario, Suscripcion, Pedido,
                                        (SELECT Pedido.id as pedido_id, Pedido.usuario_id, Empresa.id as empresa_id, Empresa.precio_despacho, SUM(Plato.precio * Plato.cantidad) as precio_comida 
                                            FROM Pedido
                                            JOIN Despacho ON Despacho.pedido_id = Pedido.id
                                            JOIN Empresa ON Despacho.empresa_id = Empresa.id
                                            JOIN ContenidoPedido ON ContenidoPedido.pedido_id = Pedido.id
                                            JOIN Plato ON ContenidoPedido.plato_id = Plato.id
                                            WHERE Pedido.estado = 'entregado a cliente' 
                                            AND Pedido.fecha BETWEEN %(fecha_inicio)s AND %(fecha_fin)s
                                            GROUP BY Pedido.id, Pedido.usuario_id, Empresa.id, Empresa.precio_despacho) as gasto_pedido_mes) 
                                        WHERE Usuario.id = gasto_pedido.usuario_id 
                                        AND Pedido.id = gasto_pedido.pedido_id 
                                        AND Suscripcion.usuario_id = Usuario.id
                                        AND Suscripcion.empresa_id NOT IN (SELECT empresa_id 
                                                                        FROM Suscripcion 
                                                                        WHERE Suscripcion.usuario_id = Usuario.id) 
                                                                        AND Suscripcion.estado = 'Vigente') 
                                        GROUP BY Usuario.email) as gasto_mensual_sin_suscripcion 
        WHERE gasto_mensual_sin_suscripcion.email_usuario = Usuario.email 
        AND Suscripcion.usuario_id = Usuario.id 
        AND Suscripcion.empresa_id = Empresa.id 
        AND Suscripcion.estado = 'Vigente' 
        AND Suscripcion.tipo = 'mensual' 
        GROUP BY Usuario.email"""

    return query, query1

#Muestre todos pedidos concretados y cancelados y el valor total de ellos
def consulta3():
    query = """SELECT Pedido.id, Pedido.estado, gasto_pedido_total.gasto_pedido
        FROM Pedido, (SELECT Pedido.id, (SUM(gasto_pedido_comida.precio_comida) + SUM(gasto_pedido_comida.precio_despacho)) as gasto_pedido
                    FROM Pedido, (SELECT Pedido.id as pedido_id, Empresa.precio_despacho, SUM(Plato.precio * ContenidoPedido.cantidad) as precio_comida 
                                            FROM Pedido
                                            JOIN Despacho ON Despacho.pedido_id = Pedido.id
                                            JOIN Empresa ON Despacho.empresa_id = Empresa.id
                                            JOIN ContenidoPedido ON ContenidoPedido.pedido_id = Pedido.id
                                            JOIN Plato ON ContenidoPedido.plato_id = Plato.id
                                            GROUP BY Pedido.id, Empresa.precio_despacho) as gasto_pedido_comida) as gasto_pedido_total
        WHERE Pedido.estado IN ('entregado a cliente', 'cliente cancela', 'delivery cancela', 'restaurante cancela')"""
    return query

#Dado un estilo de plato ingresado por el usuario (string) , muestre todas los platos con ese tipo, los restaurantes que las ofrecen y las opciones de delivery.

def consulta4():
    query = """SELECT Plato.nombre, Res_em.id_restaurant, Res_em.nombre_restaurant, Res_em.sucursal, Res_em.nombre_empresa
        FROM Plato
        JOIN PlatoRestaurant ON Plato.id = PlatoRestaurant.plato_id
        JOIN (SELECT Restaurant.id as id_restaurant, Restaurant.nombre as nombre_restaurant, Restaurant.sucursal, Empresa.nombre as nombre_empresa
                    FROM Restaurant
                    JOIN ConvenioEmpresaRestaurant ON Restaurant.id = ConvenioEmpresaRestaurant.restaurant_id
                    JOIN Empresa ON Empresa.id = ConvenioEmpresaRestaurant.empresa_id
                    WHERE Restaurant.vigente = TRUE) AS Res_em ON PlatoRestaurant.restaurant_id = Res_em.id_restaurant
        WHERE Plato.estilo = %(estilo)s 
        AND Plato.disponibilidad = TRUE"""
    
    return query

#Dado un estilo de plato ingresado (string) por el usuario, muestre todas las platos que pertenezcan a ese estilo y sus restricciones
def consulta5():
    query = """SELECT Restaurant. id, Restaurant.nombre, Restaurant.sucursal, Plato.nombre, Plato.restriccion
        FROM Plato
        JOIN Platorestaurant ON Plato.id = PlatoRestaurant.plato_id 
        JOIN Restaurant ON Platorestaurant.restaurant_id = Restaurant.id
        WHERE Plato.estilo = %(estilo)s 
        AND Plato.disponibilidad = TRUE
        AND Restaurant.vigente = TRUE"""
    return query

#Dado un cliente ingresado por el usuario (email string), muestre todas las restaurantes a las que tiene acceso con sus suscripciones.

def consulta6():
    query = """SELECT DISTINCT Restaurant.nombre, Empresa.nombre 
        FROM Restaurant 
        JOIN ConvenioEmpresaRestaurant ON Restaurant.id = ConvenioEmpresaRestaurant.restaurant_id
        JOIN Empresa ON Empresa.id = ConvenioEmpresaRestaurant.empresa_id
        JOIN Suscripcion ON Empresa.id = Suscripcion.empresa_id
        JOIN Usuario ON Suscripcion.usuario_id = Usuario.id
        WHERE Usuario.email = %(email)s"""
    return query

# Muestre la suma de dinero gastada por cada cliente en pedidos no incluidas en planes de suscripcion

def consulta7():
    query = """SELECT Usuario.email, (SUM(gasto_comida.precio_comida) + SUM(gasto_comida.precio_despacho)) as gasto_total
        FROM Usuario, Suscripcion, Pedido 
        (SELECT Pedido.id as pedido_id, Pedido.usuario_id, Empresa.id as empresa_id, Empresa.precio_despacho, SUM(Plato.precio * ContenidoPedido.cantidad) as precio_comida 
            FROM Pedido
            JOIN Despacho ON Despacho.pedido_id = Pedido.id
            JOIN Empresa ON Despacho.empresa_id = Empresa.id
            JOIN ContenidoPedido ON ContenidoPedido.pedido_id = Pedido.id
            JOIN Plato ON ContenidoPedido.plato_id = Plato.id
            WHERE Pedido.estado = 'entregado a cliente'  
            GROUP BY Pedido.id, Pedido.usuario_id, Empresa.id, Empresa.precio_despacho) as gasto_pedido
        WHERE Usuario.id = gasto_pedido.usuario_id 
        AND Pedido.id = gasto_pedido.pedido_id 
        AND Suscripcion.usuario_id = Usuario.id 
        AND Suscripcion.empresa_id NOT IN (SELECT empresa_id 
                                        FROM Suscripcion 
                                        WHERE Suscripcion.usuario_id = Usuario.id 
                                        AND Suscripcion.estado = 'Vigente') 
        GROUP BY Usuario.email"""
    return query

#Muestre todos los platos y los restaurantes que los ofrecen

def consulta8():
    query = """SELECT Plato.nombre, Restaurant.id, Restaurant.nombre, Restaurant.sucursal 
        FROM Plato
        JOIN Platorestaurant ON Plato.id = PlatoRestaurant.plato_id 
        JOIN Restaurant ON PlatoRestaurant.restaurant_id = Restaurant.id
        WHERE Plato.disponibilidad = TRUE
        AND Restaurant.vigente = TRUE"""
    return query

#Dado un numero (1-5) ingresado por el usuario, muestre todos las evaluaciones de Clientes, Delivery, Despachador superiores o iguales a  ́el

def consulta9(tipo: str):

    if tipo == 'cliente':
        query = """SELECT Pedido.id, Pedido.evaluacion_cliente, Usuario.email 
        FROM Pedido
        JOIN Usuario ON Pedido.usuario_id = Usuario.id
        WHERE Pedido.evaluacion_cliente >= %(numero)s"""
        return query
    elif tipo == 'delivery':
        query = """SELECT Pedido.id, Pedido.evaluacion_servicio as evaluacion_delivery, Empresa.nombre
        FROM Pedido 
        JOIN Despacho ON Pedido.id = Despacho.pedido_id
        JOIN Empresa ON Despacho.empresa_id = Empresa.id
        WHERE Pedido.evaluacion_servicio >= %(numero)s"""
        return query
    elif tipo == 'despachador':
        query = """SELECT Pedido.id, Pedido.evaluacion_servicio as evaluacion_despachador, Despachador.nombre
        FROM Pedido 
        JOIN Serviciodespacho ON Pedido.id = Serviciodespacho.pedido_id
        JOIN Despachador ON Serviciodespacho.despachador_id = Despachador.id
        WHERE Pedido.evaluacion_servicio >= %(numero)s"""
        return query

#Dado una un alergeno ingresado por el usuario (string), muestre todos los platos que lo contengan en sus ingredientes (Ejemplo: maní)
def consulta10():
    query = """SELECT Restaurant.id, Restaurant.nombre as nombre_restaurant, Restaurant.sucursal, Plato.nombre as nombre_plato 
        FROM Plato
        JOIN Platorestaurant ON Plato.id = Platorestaurant.plato_id 
        JOIN Restaurant ON Platorestaurant.restaurant_id = Restaurant.id
        WHERE Plato.ingredientes LIKE %(alergeno)s 
        AND Plato.disponibilidad = TRUE"""
    return query

In [4]:
def consultas(numero_consulta, **kwargs):
    conn = None
    cursor = None
    
    try:
        # Conexión con la base de datos
        conn = psycopg2.connect(
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME
        )
        # Crear un cursor para ejecutar consultas
        cursor = conn.cursor()

        if numero_consulta == 1:
            try:
                query = consulta1()
                nombre_plato = kwargs.get('nombre_plato')
                cursor.execute(query, {'nombre': nombre_plato})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 2:
            try:
                query, query1 = consulta2()
                email = kwargs.get('email')
                fecha_inicio = kwargs.get('fecha_inicio')
                fecha_fin = kwargs.get('fecha_fin')
                cursor.execute(query, {'email': email})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
                cursor.execute(query1, {'fecha_inicio': fecha_inicio, 'fecha_fin': fecha_fin})
                rows = cursor.fetchall()
                print(rows)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 3:
            try:
                query = consulta3()
                cursor.execute(query)
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 4:
            try:
                query = consulta4()
                estilo = kwargs.get('estilo')
                cursor.execute(query, {'estilo': estilo})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 5:
            try:
                query = consulta5()
                estilo = kwargs.get('estilo')
                cursor.execute(query, {'estilo': estilo})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 6:
            try:
                query = consulta6()
                email = kwargs.get('email')
                cursor.execute(query, {'email': email})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 7:
            try:
                query = consulta7()
                cursor.execute(query)
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 8:
            try:
                query = consulta8()
                cursor.execute(query)
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)
        
        if numero_consulta == 9:
            try:
                tipo = kwargs.get('tipo')
                query = consulta9(tipo)
                numero = kwargs.get('numero')
                cursor.execute(query, {'numero': numero})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 10:
            try:
                query = consulta10()
                alergeno = kwargs.get('alergeno')
                cursor.execute(query, {'alergeno': '%{}%'.format(alergeno)}) 
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

    except (Exception, Error) as error:
        print("Error al conectarse al servidor:", error)
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [5]:
consultas(1, nombre_plato='Pizza Margarita')

(162, 'Just Pizza', 'Maintenci\x97n')
(163, 'Just Pizza', 'Grecia')
(164, 'Just Pizza', 'Estado')
(165, 'Just Pizza', 'Apoquindo')
(166, 'Just Pizza', 'Viel')
(167, 'Just Pizza', 'OHiggins')
(168, 'Just Pizza', 'Prat')
(169, 'Just Pizza', 'Alemania')
(170, 'Just Pizza', 'Larga')
(171, 'Just Pizza', 'Col\x97n')
(85, 'Doble Queso', 'Estado')
(86, 'Doble Queso', 'Apoquindo')
(87, 'Doble Queso', 'Viel')
(88, 'Doble Queso', 'OHiggins')
(89, 'Doble Queso', 'Prat')
(90, 'Doble Queso', 'Alemania')
(85, 'Doble Queso', 'Estado')
(86, 'Doble Queso', 'Apoquindo')
(87, 'Doble Queso', 'Viel')
(88, 'Doble Queso', 'OHiggins')
(89, 'Doble Queso', 'Prat')
(90, 'Doble Queso', 'Alemania')
(16, 'Mestiza', 'Lientur')


In [6]:
consultas(5, estilo='Internacional')

(111, 'DeliQuick', 'San Antonio', 'Spaghetti Carbonara', 'alergicos')
(110, 'DeliQuick', 'Apoquindo', 'Spaghetti Carbonara', 'alergicos')
(109, 'DeliQuick', 'Vitacura', 'Spaghetti Carbonara', 'alergicos')
(108, 'DeliQuick', 'Col\x97n', 'Spaghetti Carbonara', 'alergicos')
(24, 'Chick-Fil-A', 'Alemania', 'Tacos al Pastor', 'celiacos')
(23, 'Chick-Fil-A', 'Prat', 'Tacos al Pastor', 'celiacos')
(22, 'Chick-Fil-A', 'OHiggins', 'Tacos al Pastor', 'celiacos')
(21, 'Chick-Fil-A', 'Viel', 'Tacos al Pastor', 'celiacos')
(20, 'Chick-Fil-A', 'Grecia', 'Tacos al Pastor', 'celiacos')
(19, 'Chick-Fil-A', 'Estado', 'Tacos al Pastor', 'celiacos')
(18, 'Chick-Fil-A', 'Apoquindo', 'Tacos al Pastor', 'celiacos')
(76, 'Krusty Krab', 'Col\x97n', 'Risotto', 'pescetarianos')
(75, 'Krusty Krab', 'Larga', 'Risotto', 'pescetarianos')
(74, 'Krusty Krab', 'Alemania', 'Risotto', 'pescetarianos')
(73, 'Krusty Krab', 'Prat', 'Risotto', 'pescetarianos')
(72, 'Krusty Krab', 'OHiggins', 'Risotto', 'pescetarianos')
(71, 

In [7]:
consultas(8)

('DCCHamburguer', 1, 'DCCRestaurant', 'Las Parcelas')
('Pizza Margarita', 171, 'Just Pizza', 'Col\x97n')
('Pizza Margarita', 170, 'Just Pizza', 'Larga')
('Pizza Margarita', 169, 'Just Pizza', 'Alemania')
('Pizza Margarita', 168, 'Just Pizza', 'Prat')
('Pizza Margarita', 167, 'Just Pizza', 'OHiggins')
('Pizza Margarita', 166, 'Just Pizza', 'Viel')
('Pizza Margarita', 165, 'Just Pizza', 'Apoquindo')
('Pizza Margarita', 164, 'Just Pizza', 'Estado')
('Pizza Margarita', 163, 'Just Pizza', 'Grecia')
('Pizza Margarita', 162, 'Just Pizza', 'Maintenci\x97n')
('Pizza Margarita', 90, 'Doble Queso', 'Alemania')
('Pizza Margarita', 89, 'Doble Queso', 'Prat')
('Pizza Margarita', 88, 'Doble Queso', 'OHiggins')
('Pizza Margarita', 87, 'Doble Queso', 'Viel')
('Pizza Margarita', 86, 'Doble Queso', 'Apoquindo')
('Pizza Margarita', 85, 'Doble Queso', 'Estado')
('Pizza Margarita', 90, 'Doble Queso', 'Alemania')
('Pizza Margarita', 89, 'Doble Queso', 'Prat')
('Pizza Margarita', 88, 'Doble Queso', 'OHiggins'

In [8]:
consultas(10, alergeno = 'ketchup')

(2, 'Meet Burger', 'Providencia', 'Hamburguesa con Queso')
(3, 'Meet Burger', 'Vitacura', 'Hamburguesa con Queso')
(4, 'Meet Burger', 'Apoquindo', 'Hamburguesa con Queso')
(5, 'Meet Burger', 'Kennedy', 'Hamburguesa con Queso')
(1, 'DCCRestaurant', 'Las Parcelas', 'Hamburguesa con Queso')
(25, 'Donald Mc', 'Grecia', 'Hamburguesa con Queso')
(26, 'Donald Mc', 'Estado', 'Hamburguesa con Queso')
(27, 'Donald Mc', 'Apoquindo', 'Hamburguesa con Queso')
(28, 'Donald Mc', 'Viel', 'Hamburguesa con Queso')
(29, 'Donald Mc', 'OHiggins', 'Hamburguesa con Queso')
(30, 'Donald Mc', 'Prat', 'Hamburguesa con Queso')
(31, 'Donald Mc', 'Alemania', 'Hamburguesa con Queso')
(32, 'Donald Mc', 'Larga', 'Hamburguesa con Queso')
(33, 'Donald Mc', 'Col\x97n', 'Hamburguesa con Queso')
(34, 'Donald Mc', 'Vitacura', 'Hamburguesa con Queso')
(85, 'Doble Queso', 'Estado', 'Hamburguesa con Queso')
(86, 'Doble Queso', 'Apoquindo', 'Hamburguesa con Queso')
(87, 'Doble Queso', 'Viel', 'Hamburguesa con Queso')
(88, 'Dob

In [21]:
consultas(4, estilo='Internacional')

('Jambalaya', 1, 'DCCRestaurant', 'Las Parcelas', 'ATuCasa')
('Fajitas', 1, 'DCCRestaurant', 'Las Parcelas', 'ATuCasa')
('Dim Sum', 1, 'DCCRestaurant', 'Las Parcelas', 'ATuCasa')
('Jambalaya', 1, 'DCCRestaurant', 'Las Parcelas', 'FoodBasics')
('Fajitas', 1, 'DCCRestaurant', 'Las Parcelas', 'FoodBasics')
('Dim Sum', 1, 'DCCRestaurant', 'Las Parcelas', 'FoodBasics')
('Chiles Rellenos', 67, 'Krusty Krab', 'Maintenci\x97n', 'FoodBasics')
('Risotto', 67, 'Krusty Krab', 'Maintenci\x97n', 'FoodBasics')
('Chiles Rellenos', 68, 'Krusty Krab', 'Grecia', 'FoodBasics')
('Risotto', 68, 'Krusty Krab', 'Grecia', 'FoodBasics')
('Chiles Rellenos', 69, 'Krusty Krab', 'Estado', 'FoodBasics')
('Risotto', 69, 'Krusty Krab', 'Estado', 'FoodBasics')
('Chiles Rellenos', 70, 'Krusty Krab', 'Apoquindo', 'FoodBasics')
('Risotto', 70, 'Krusty Krab', 'Apoquindo', 'FoodBasics')
('Chiles Rellenos', 71, 'Krusty Krab', 'Viel', 'FoodBasics')
('Risotto', 71, 'Krusty Krab', 'Viel', 'FoodBasics')
('Chiles Rellenos', 72, '

In [22]:
consultas(6, email = 'Pedro.Correa@bbdduc.utopia')

('Bogaro', 'DCCRappi')
('Bogaro', 'EsquinaShop')
('Bogaro', 'PedidosNow')
('Burger Queen', 'ATuCasa')
('Burger Queen', 'DCCRappi')
('Burger Queen', 'EsquinaShop')
('Burger Queen', 'PedidosNow')
('Burger Queen', 'Speed')
('Carlos Sr', 'FoodBasics')
('Casiss', 'ATuCasa')
('Casiss', 'DCCRappi')
('Casiss', 'FoodBasics')
('Casiss', 'PedidosNow')
('Central Perk', 'DCCRappi')
('Central Perk', 'EsquinaShop')
('Central Perk', 'Speed')
('Chick-Fil-A', 'ATuCasa')
('Chick-Fil-A', 'FoodBasics')
('Chick-Fil-A', 'PedidosNow')
('Chick-Fil-A', 'Speed')
('Chum Bucket', 'ATuCasa')
('Chum Bucket', 'DCCRappi')
('Chum Bucket', 'EsquinaShop')
('Comida Peruana', 'EsquinaShop')
('Comida Peruana', 'FoodBasics')
('Comida Peruana', 'Speed')
('DCCRestaurant', 'ATuCasa')
('DCCRestaurant', 'DCCRappi')
('DCCRestaurant', 'EsquinaShop')
('DCCRestaurant', 'FoodBasics')
('DCCRestaurant', 'Speed')
('Delicia China', 'FoodBasics')
('DeliQuick', 'DCCRappi')
('DeliQuick', 'EsquinaShop')
('DeliQuick', 'Speed')
('Doble Queso', 

In [25]:
consultas(9, numero = 4, tipo = 'cliente')

(54, 4, 'Ignacio.Arias@bbdduc.utopia')
(100, 4, 'Antonio.Moreno@bbdduc.utopia')
(57, 4, 'Macarena.Saldana@bbdduc.utopia')
(76, 5, 'Alejandro.Sierra@bbdduc.utopia')
(59, 4, 'Yan.Zhung@bbdduc.utopia')
(84, 4, 'Jesus.Ramirez@bbdduc.utopia')
(33, 4, 'Ozzy.Baldini@bbdduc.utopia')
(52, 5, 'Pedro.Correa@bbdduc.utopia')
(64, 5, 'Andres.Mendez@bbdduc.utopia')
(19, 4, 'Andrea.Guevara@bbdduc.utopia')
(22, 5, 'Ignacio.Conejeros@bbdduc.utopia')
(25, 4, 'Alejandro.Daroca@bbdduc.utopia')
(95, 4, 'Andres.Ramirez@bbdduc.utopia')
(5, 5, 'Andres.Sanchez@bbdduc.utopia')
(35, 5, 'Javier.Madrid@bbdduc.utopia')
(17, 4, 'Nicolas.Plaza@bbdduc.utopia')
(37, 4, 'Enrique.Costa@bbdduc.utopia')
(69, 5, 'Agustin.Reyes@bbdduc.utopia')
(13, 5, 'Agustin.Matte@bbdduc.utopia')
(28, 4, 'Antonio.Larrain@bbdduc.utopia')
(21, 4, 'Humberto.Hormann@bbdduc.utopia')
(78, 5, 'Ignacia.Ruiz@bbdduc.utopia')
(27, 4, 'Andres.Barrera@bbdduc.utopia')
(101, 5, 'Enoc.Asenjo@bbdduc.utopia')
(81, 5, 'Nicolas.Ayala@bbdduc.utopia')
(67, 5, 'A